## Preprocess

In [1]:
import numpy as np 
import pandas as pd

In [2]:
x_transaction = pd.read_csv('../../kkbox-churn-prediction-challenge/under_sample/X_train_transactions.csv')

In [3]:
x_transaction.head(1)

,Unnamed: 0,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,27,LdLM57UzJ9+j/hwiTzzqd2Zi4AbSpFw93q2S77dZ8q0=,37,30,149,149,1,20150902,20151002,0


In [4]:
x_transaction=x_transaction.drop(['Unnamed: 0'], axis=1)

In [5]:
x_transaction.shape

(441385, 9)

In [6]:
x_transaction['payment_plan_days'].unique()

array([ 30,  31,   0, 195,   7, 410, 395, 120, 180, 100,  60,   1,  90,
       400,  10, 450,  14, 240,  70, 360, 200,  35,  99,  80,  66, 365,
       230,  21, 270,   3,   2, 110,  15])

### Variables generated from plan_list_price & actual_amount_paid
1. discount_amount = plan_list_price - actual_amount_paid
There is negative discount_amount, therefore, check each of the situation, get the following:

7 days: 35

30 days: 149/129/119/150/100/134/131

60 days: 300

80 days: 400

90 days: 480/447/450

180 days = 894/930/799/536

365 days = 1788

2. Current method:

a) If plan_list_price = 0, plan_list_price=actual_amount_paid and assgin payment_plan_days by checking actual_amount_paid

b) If actual_amount_paid=0, get a new column called free

c) Discount = plan_list_price - actual_amount_paid

3. Problem of this method

The negative list may be more with the amount of data incereasing. But seems inappropriate to delete them because it usually stands abnormal situations.

In [7]:
# get the discount_amount
x_transaction['discount_amount'] = x_transaction['plan_list_price'] - x_transaction['actual_amount_paid']

In [8]:
x_transaction['discount_amount'].unique()

array([    0,  -149,    30,  -129,   149,  -119,  -300,   180,  -150,
        -100,  -894,    20, -1788,   -35,  -134,    50,  -480,  -131,
        -930,  -400,  -447,   120,  -450,  -799,  -536])

In [9]:
# the negative ones
len(x_transaction[x_transaction['discount_amount']<0])

18016

In [10]:
# change "plan_list_price"
x_transaction['plan_list_price'] = np.where(x_transaction['discount_amount']<0, x_transaction['actual_amount_paid'],x_transaction['plan_list_price'])

In [11]:
# change "payment_plan_days"
conditions = [
    (x_transaction['discount_amount'] ==-35),
    (x_transaction['discount_amount'].isin([-149,-129,-119,-150,-100,-134,-131])),
    (x_transaction['discount_amount'] ==-300),
    (x_transaction['discount_amount'] ==-400),
    (x_transaction['discount_amount'].isin([-480,-447,-450])),
    (x_transaction['discount_amount'].isin([-894,-930,-799,-536])),
    (x_transaction['discount_amount'] ==-1788)]
choices = [7, 30, 60,80,90,180,365]
x_transaction['payment_plan_days'] = np.select(conditions,choices,x_transaction['payment_plan_days'])

In [12]:
(x_transaction[x_transaction['discount_amount']<0].payment_plan_days>0).unique()

array([ True])

In [13]:
# change the negative discount to be 0
x_transaction['discount_amount'] = np.where(x_transaction['discount_amount']<0, 0, x_transaction['discount_amount'])

In [14]:
# get a new column called free
x_transaction['free']=0
x_transaction['free'] = np.where(x_transaction['actual_amount_paid']==0, 1, x_transaction['free'])

### Variables related to date

In [15]:
x_transaction=x_transaction.sort_values(by=['msno', 'transaction_date'])
x_transaction.head(1)

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,discount_amount,free
274896,++DcyRE+ZfLtlKGigvv9dv5EQ4KAhHhiN9LM9X+F1vw=,37,31,149,149,1,20150123,20150224,0,0,0


In [16]:
# days between transaction & expiration
x_transaction.membership_expire_date=pd.to_datetime(x_transaction.membership_expire_date, format="%Y%m%d")
x_transaction.transaction_date=pd.to_datetime(x_transaction.transaction_date, format="%Y%m%d")
x_transaction['length']=(x_transaction['membership_expire_date']-x_transaction['transaction_date']).dt.days

In [17]:
x_transaction.head(2)

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,discount_amount,free,length
274896,++DcyRE+ZfLtlKGigvv9dv5EQ4KAhHhiN9LM9X+F1vw=,37,31,149,149,1,2015-01-23,2015-02-24,0,0,0,32
274893,++DcyRE+ZfLtlKGigvv9dv5EQ4KAhHhiN9LM9X+F1vw=,37,31,149,149,1,2015-02-23,2015-03-24,0,0,0,29


In [18]:
# the average amount paid by user in each payment
x_transaction['amtperday']=x_transaction['actual_amount_paid']/x_transaction['payment_plan_days']

### Sort the table to get information from records between each trasanction for a user

In [19]:
# get the consecutive difference from rows
x_transaction["d_transaction"] = x_transaction["transaction_date"].diff(1).fillna(0).astype('timedelta64[D]').astype(int)
x_transaction.d_transaction = np.where(x_transaction.d_transaction < 0, 0,x_transaction.d_transaction)

In [20]:
# same for expire_date
x_transaction["d_expire_date"] = x_transaction["membership_expire_date"].diff(1).fillna(0).astype('timedelta64[D]').astype(int)
x_transaction.d_expire_date = np.where(x_transaction.d_expire_date < 0, 0,x_transaction.d_expire_date)

### A potential sign of churn (auto_renew=0 & expiration in 201702)

In [21]:
from datetime import datetime
startdate = datetime(2017, 2, 1)
enddate = datetime(2017, 2, 28)
x_transaction['potential_churn'] = np.where((x_transaction['is_auto_renew']==0)&(x_transaction['membership_expire_date']>=startdate) 
                                            & (x_transaction['membership_expire_date'] <= enddate), 1,0)

In [22]:
x_transaction.head(1)

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,discount_amount,free,length,amtperday,d_transaction,d_expire_date,potential_churn
274896,++DcyRE+ZfLtlKGigvv9dv5EQ4KAhHhiN9LM9X+F1vw=,37,31,149,149,1,2015-01-23,2015-02-24,0,0,0,32,4.806452,0,0,0


### Groupby Customer and generate related variables

In [23]:
# define some common used funcitons
most_common = lambda x: pd.Series.mode(x)[0]
most_common.__name__ = 'most_common'
max_min_diff = lambda x: x.max() - x.min()
max_min_diff.__name__ = 'max_min_diff'
def pcet_of_zero(x):
    return 1-(x.mean())
def change_or_not(x):
    return (x.nunique()-1)
def find_positive_pct(x):
    return ((x>0).sum()/x.count())
def has_discount(x):
    return x.nunique()>1

In [24]:
# def change column name
def chagne_name(df):
    table = df.columns.levels[0][0]+"_"
    cols=df.columns.levels[1]
    cols=cols[-1:]+cols[:-1]
    cols=["{}".format(table) + s for s in cols]
    cols.insert(0, "msno")
    return cols

# def change column name 2:
def change_name_2(df):
    df.columns = ["_".join(x) for x in df.columns.ravel()]
    df.rename(columns={'msno_':'msno'}, inplace=True)
    return df

In [25]:
# payment_method
payment_method=x_transaction.groupby('msno',as_index=False).agg({
    'payment_method_id':['nunique',most_common]})
payment_method.columns=chagne_name(payment_method)

In [26]:
payment_method.head(1)

,msno,payment_method_id_nunique,payment_method_id_most_common
0,++DcyRE+ZfLtlKGigvv9dv5EQ4KAhHhiN9LM9X+F1vw=,1,37


In [27]:
# payment_plan_days
payment_plan_days=x_transaction.groupby('msno',as_index=False).agg({
    'payment_plan_days':['nunique',most_common]})
payment_plan_days.columns=chagne_name(payment_plan_days)

In [28]:
payment_plan_days.head(1)

,msno,payment_plan_days_nunique,payment_plan_days_most_common
0,++DcyRE+ZfLtlKGigvv9dv5EQ4KAhHhiN9LM9X+F1vw=,2,30


In [29]:
# plan_list_price
plan_list_price=x_transaction.groupby('msno',as_index=False).agg({
    'plan_list_price':['nunique', most_common,'mean',max_min_diff]})
plan_list_price.columns=chagne_name(plan_list_price)

In [30]:
plan_list_price.head(1)

,msno,plan_list_price_nunique,plan_list_price_most_common,plan_list_price_mean,plan_list_price_max_min_diff
0,++DcyRE+ZfLtlKGigvv9dv5EQ4KAhHhiN9LM9X+F1vw=,1,149,149.0,0


In [31]:
# actual_amount_paid
actual_amount_paid=x_transaction.groupby('msno',as_index=False).agg({
    'actual_amount_paid':['nunique', most_common,'mean',max_min_diff]})
actual_amount_paid.columns=chagne_name(actual_amount_paid)

In [32]:
actual_amount_paid.head(1)

,msno,actual_amount_paid_nunique,actual_amount_paid_most_common,actual_amount_paid_mean,actual_amount_paid_max_min_diff
0,++DcyRE+ZfLtlKGigvv9dv5EQ4KAhHhiN9LM9X+F1vw=,1,149,149.0,0


In [33]:
# is_auto_renew
is_auto_renew=x_transaction.groupby('msno',as_index=False).agg({
    'is_auto_renew':[pcet_of_zero, change_or_not,most_common]})
is_auto_renew.columns=chagne_name(is_auto_renew)

In [34]:
# is_cancel
is_cancel=x_transaction.groupby('msno',as_index=False).agg({
    'is_cancel':['mean', change_or_not]})
is_cancel.columns=chagne_name(is_cancel)

In [35]:
is_cancel.head(1)

,msno,is_cancel_mean,is_cancel_change_or_not
0,++DcyRE+ZfLtlKGigvv9dv5EQ4KAhHhiN9LM9X+F1vw=,0.038462,1


In [36]:
# discount_amount
discount_amount=x_transaction.groupby('msno',as_index=False).agg({
    'discount_amount':[find_positive_pct, 'mean','sum',has_discount]})
discount_amount.columns=chagne_name(discount_amount)

In [37]:
discount_amount.head(1)

,msno,discount_amount_find_positive_pct,discount_amount_mean,discount_amount_sum,discount_amount_has_discount
0,++DcyRE+ZfLtlKGigvv9dv5EQ4KAhHhiN9LM9X+F1vw=,0.0,0.0,0,False


In [38]:
# free
free=x_transaction.groupby('msno',as_index=False).agg({
    'free':['mean',has_discount]})
free.columns=chagne_name(free)

In [39]:
free.columns

Index(['msno', 'free_mean', 'free_has_discount'], dtype='object')

In [40]:
# length
length=x_transaction.groupby('msno',as_index=False).agg({
    'length':['mean','sum','std','first']}).fillna(0)

In [41]:
# another rename method
length=change_name_2(length)

In [42]:
length.head(1)

,msno,length_mean,length_sum,length_std,length_first
0,++DcyRE+ZfLtlKGigvv9dv5EQ4KAhHhiN9LM9X+F1vw=,29.423077,765,6.047631,32


In [43]:
# amtperday
amtperday=x_transaction.groupby('msno',as_index=False).agg({
    'amtperday':['mean']}).fillna(0)
amtperday=change_name_2(amtperday)

In [44]:
amtperday.head(1)

,msno,amtperday_mean
0,++DcyRE+ZfLtlKGigvv9dv5EQ4KAhHhiN9LM9X+F1vw=,4.94818


In [45]:
# transaction_date
transaction_date=x_transaction.groupby('msno',as_index=False).agg({
    'transaction_date':['first']})
transaction_date=change_name_2(transaction_date)

In [46]:
# d_transaction
d_transaction=x_transaction.groupby('msno',as_index=False).agg({
    'd_transaction':['mean','std']}).fillna(0)
d_transaction=change_name_2(d_transaction)

In [47]:
d_transaction.head(1)

,msno,d_transaction_mean,d_transaction_std
0,++DcyRE+ZfLtlKGigvv9dv5EQ4KAhHhiN9LM9X+F1vw=,29.346154,6.039485


In [48]:
# d_expire_date
d_expire_date=x_transaction.groupby('msno',as_index=False).agg({
    'd_expire_date':['mean','std','sum']}).fillna(0)
d_expire_date=change_name_2(d_expire_date)

In [49]:
d_expire_date.head(1)

,msno,d_expire_date_mean,d_expire_date_std,d_expire_date_sum
0,++DcyRE+ZfLtlKGigvv9dv5EQ4KAhHhiN9LM9X+F1vw=,28.115385,8.310605,731


In [50]:
# potential_churn
potential_churn=x_transaction.groupby('msno',as_index=False).agg({
    'potential_churn':[has_discount]})
potential_churn.columns=chagne_name(potential_churn)

In [51]:
potential_churn.head(1)

,msno,potential_churn_has_discount
0,++DcyRE+ZfLtlKGigvv9dv5EQ4KAhHhiN9LM9X+F1vw=,False


In [52]:
data_frames=[payment_method,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,is_cancel,discount_amount,free,length,amtperday,transaction_date,d_transaction,d_expire_date,potential_churn]

In [53]:
from functools import reduce
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['msno'],
                                            how='inner'), data_frames)

In [59]:
df_merged['membership_days']=df_merged['d_expire_date_sum']+df_merged['length_first'] 
# get total membership days
df_merged=df_merged.drop(['d_expire_date_sum', 'length_first'], axis=1)

In [60]:
df_merged.to_csv('../../kkbox-churn-prediction-challenge/train/X_train_transactions.csv',index=False)

In [61]:
df = pd.read_csv('../../kkbox-churn-prediction-challenge/train/X_train_transactions.csv')

In [62]:
df.shape

(24798, 35)